In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

N = 5000             
g = 1.5               
phi = torch.tanh
dt = 0.1
steps = 3000
num_trials = 200
steady_check_interval = 10   
steady_threshold = 1e-5      
avg_window = 10              

def check_steady(kinetic_history,window=3,threshold=1e-5):
    if len(kinetic_history) < window *2 :
        return False
    recent = np.mean(kinetic_history[-window:])
    pre = np.mean(kinetic_history[-2*window+1:-window])
    return abs(recent - pre) < threshold

def Classical_rnn_energy(J, num_trials=num_trials):
    kinetic_list = []
    for _ in range(num_trials):
        x = torch.randn(N, device=device)
        kinetic_history = []
        steady_reached = False
        buffer = []  # 用于计算最近10步动能平均
        for t in range(steps):
            dx = -x + J @ phi(x)
            x = x + dt * dx
            buffer.append((dx**2).mean().item()/2)
            if t>1000 and (t + 1) % steady_check_interval == 0:
                mean_kin = np.mean(buffer[-avg_window:])
                kinetic_history.append(mean_kin)
                if check_steady(kinetic_history, threshold=steady_threshold):
                    steady_reached = True
                    break
        if steady_reached:
            for _ in range(200):
                dx = -x + J @ phi(x)
                x = x + dt * dx
                kinetic_list.append((dx**2).mean().item()/2)
    return np.mean(kinetic_list),np.std(kinetic_list)

def Langevin_dynamics_energy(J, T, num_trials=num_trials):
    kinetic_list = []
    for _ in range(num_trials):
        x = torch.randn(N, device=device)
        kinetic_history = []
        steady_reached = False
        buffer = []
        for t in range(steps):
            x = x.detach().requires_grad_(True)
            inner = -x + J @ phi(x)
            E = 0.5 * (inner**2).sum()
            gradE, = torch.autograd.grad(E, x)
            noise = torch.sqrt(torch.tensor(2*T*dt, device=device)) * torch.randn_like(x)
            dx = -gradE * dt + noise
            x = x + dx
            buffer.append((dx**2).mean().item()/2)
            if t>1000 and (t + 1) % steady_check_interval == 0:
                mean_kin = np.mean(buffer[-avg_window:])
                kinetic_history.append(mean_kin)
                if check_steady(kinetic_history, threshold=steady_threshold):
                    steady_reached = True
                    break
        if steady_reached:
            for _ in range(200):
                x = x.detach().requires_grad_(True)
                inner = -x + J @ phi(x)
                E = 0.5 * (inner**2).sum()
                gradE, = torch.autograd.grad(E, x)
                noise = torch.sqrt(torch.tensor(2*T*dt, device=device)) * torch.randn_like(x)
                dx = -gradE * dt + noise
                x = x + dx
                kinetic_list.append((dx**2).mean().item()/2)
    return np.mean(kinetic_list),np.std(kinetic_list)

df = pd.read_csv("Teff.csv")
Te = df['T_eff']
torch.manual_seed(1997)
J = torch.randn(N, N, device=device) * g / np.sqrt(N)
E_rnn ,E_rnn_std= Classical_rnn_energy(J, num_trials=num_trials)
print(f"Average kinetic energy (classical RNN): {E_rnn:.5f}")

def find_effective_temperature(J, target_energy, T_low=0.05, T_high=0.2, tol=1e-8, max_iter=10):
    for i in range(max_iter):
        T_mid = 0.5 * (T_low + T_high)
        E_mid = Langevin_dynamics_energy(J, T_mid)
        print(f"Iter {i+1}: T_mid={T_mid:.6f}, E_mid={E_mid:.6f}")
        if abs(E_mid - target_energy) < tol:
            return T_mid, E_mid
        if E_mid > target_energy:
            T_high = T_mid
        else:
            T_low = T_mid
    return T_mid, E_mid

T_eff = Te[1]
print(f"\nEstimated effective temperature: T_eff = {T_eff:.6f}")

T_list = torch.linspace(0.0001, 0.2, 20)
E_GD_list = []
E_GD_std = []
for T in tqdm(T_list, desc="Scanning temperature"):
    E_boltz ,E_boltz_std= Langevin_dynamics_energy(J, T.item())
    E_GD_list.append(E_boltz)
    E_GD_std.append(E_boltz_std)
E_boltz_list = np.array(E_GD_list)
E_boltz_std = np.array(E_GD_std)

df = pd.DataFrame({
    "Temperature": T_list.cpu().numpy(),
    "GDE": E_boltz_list,
    "GDE_std": E_boltz_std,
    "T_eff": T_eff,
    'E_rnn_value': E_rnn,
    'E_rnn_std': E_rnn_std
})
df.to_csv("energy_vs_T.csv", index=False)
print("✅ Data saved to energy_vs_T.csv")

with open("E_rnn_value.txt", "w") as f:
    f.write(str(E_rnn))
plt.figure(figsize=(6,4))
plt.plot(T_list.cpu(), E_boltz_list, 'o-', label='Boltzmann dynamics')
plt.axhline(E_rnn, color='r', linestyle='--', label=f'Classical, E={E_rnn:.5f}')
plt.axvline(T_eff, color='g', linestyle=':', label=f'T_eff={T_eff:.5f}')
plt.xlabel('Temperature T')
plt.ylabel('Average kinetic energy')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import os
save_dir = os.getcwd()
print("Saving data to:", save_dir)

torch.manual_seed(1997)
J = torch.randn(N, N, device=device) * g / np.sqrt(N)

def check_steady(kinetic_history, window=3, threshold=1e-5):
    if len(kinetic_history) < 2 * window:
        return False
    recent = np.mean(kinetic_history[-window:])
    prev = np.mean(kinetic_history[-2 * window:-window])
    return abs(recent - prev) < threshold

Steps = 2400
def sample_boltzmann(J, T, num_trials=num_trials):
    traj = []
    for _ in range(num_trials):
        kinetic_history = []
        buffer = []
        steady_reached = False
        x = torch.randn(N, device=device)
        for t in range(steps):
            x = x.detach().requires_grad_(True)
            inner = -x + J @ phi(x)
            E = 0.5 * (inner**2).sum()
            gradE, = torch.autograd.grad(E, x, create_graph=False)
            noise = torch.sqrt(torch.tensor(2*T*dt, device=device)) * torch.randn_like(x)
            dx = -gradE * dt + noise
            x = x + dx
            buffer.append((dx**2).mean().item()/2)
            if (t + 1) % 10 == 0:
                mean_kin = np.mean(buffer[-10:])
                kinetic_history.append(mean_kin)
                if check_steady(kinetic_history, threshold=1e-5):
                    steady_reached = True
                    break
    
        if steady_reached:
            for _ in range(200):
                x = x.detach().requires_grad_(True)
                inner = -x + J @ phi(x)
                E = 0.5 * (inner**2).sum()
                gradE, = torch.autograd.grad(E, x, create_graph=False)
                noise = torch.sqrt(torch.tensor(2*T*dt, device=device)) * torch.randn_like(x)
                dx = -gradE * dt + noise
                x = x + dx
                traj.append(x.detach().cpu().numpy())

    return np.array(traj)


def Classical_sample_energy(J, num_trials=num_trials):
    samples_rnn = []
    for _ in range(num_trials):
        x = torch.randn(N, device=device)
        kinetic_history = []
        buffer = []
        steady_reached = False
        for t in range(steps):
            dx = -x + J @ phi(x)
            x = x + dt * dx
            buffer.append((dx**2).mean().item()/2)
            if (t + 1) % 10 == 0:
                mean_kin = np.mean(buffer[-10:])
                kinetic_history.append(mean_kin)
                if check_steady(kinetic_history, threshold=1e-5):
                    steady_reached = True
                    break
        if steady_reached:
            for _ in range(200): 
                dx = -x + J @ phi(x)
                x = x + dt * dx
                samples_rnn.append(x.detach().cpu().numpy())
    return np.array(samples_rnn)


T1 =T_eff
print(f"T_eff: {T_eff:.3f}, T1: {T1:.3f}")
samples_boltz = sample_boltzmann(J, T1)
samples_classi = Classical_sample_energy(J, num_trials=num_trials)
samples_b = samples_boltz.reshape(-1)
samples_r = samples_classi.reshape(-1)

df_boltz = pd.DataFrame({ "model": "Boltzmann", "x": samples_b })
df_rnn = pd.DataFrame({ "model": "RNN", "x": samples_r})

pd.DataFrame({"x": samples_b}).to_csv( os.path.join(save_dir, "samples_boltz.csv"), index=False )
pd.DataFrame({"x": samples_r}).to_csv( os.path.join(save_dir, "samples_classical.csv"), index=False )

plt.figure(figsize=(6,4))
plt.hist(samples_r, bins=50, density=True, color='skyblue', alpha=0.6,label='Classical RNN')
plt.hist(samples_b, bins=50, density=True, color='pink',alpha=0.6,  label=f'Boltzmann (T_eff={T1:.3f})')
plt.xlabel('x')
plt.ylabel('P(x)')
plt.legend()
plt.tight_layout()
plt.show()


